In [1]:
import cptdl as dl 
import cptio as io 
import src as xc 
import datetime as dt
import xarray as xr
import cartopy.crs as ccrs
import warnings 
from pathlib import Path
import matplotlib.pyplot as plt 
from dask.distributed import Client 
from dask.diagnostics import ProgressBar
client = None

# Settings

In [2]:
predictor_names = ['SPSv3p5.PRCP', 'SEAS5.PRCP', 'GCFS2p1.PRCP']
predictand_name = 'UCSB.PRCP'

# use dl.observations.keys() to see all options for predictand 
# and dl.hindcasts.keys() to see all options for predictors
# make sure your first_year & final_year are compatible with 
# your selections for your predictors and predictands 

args = { 
  'fdate': dt.datetime(2022, 5, 1),
  'first_year': 1993, 
  'final_year': 2016, 
  'predictor_extent': {
    'east': 50,
    'west': -20, 
    'north': 45, 
    'south': -45
  }, 
  'predictand_extent': {
    'east': 50,
    'west': -20, 
    'north': 45, 
    'south': -45
  }, 
  'lead_low': 1.5,
  'lead_high': 3.5, 
  'target': 'Jun-Aug',
  'filetype': 'cptv10.tsv'
}

force_download = True

# Data Download

In [ ]:
# download predictand
if not Path('fewsnet_y_global.nc').is_file() or force_download:
    Y = dl.download(dl.observations[predictand_name], predictand_name +'.tsv', **args, verbose=True, use_dlauth=False)
    Y = getattr(Y, [i for i in Y.data_vars][0])
    Y = Y.expand_dims({'C': [predictand_name]})
    Y.to_netcdf('fewsnet_y_global.nc')
else:
    Y = xr.open_dataset('fewsnet_y_global.nc')
    Y = getattr(Y, [i for i in Y.data_vars][0])

# download training data 
if not Path('fewsnet_x_global.nc').is_file() or force_download:
    X = [ dl.download(dl.hindcasts[i], i+'.tsv', **args, verbose=True, use_dlauth=False) for i in predictor_names]
    X2 = []
    for i in X:
        i = getattr(i, [j for j in i.data_vars][0])
        i = xc.regrid(i.expand_dims({'M': [0]}), Y.coords['X'].values, Y.coords['Y'].values)
        X2.append(i)
    X = xr.concat(X2, 'M').assign_coords({'M': predictor_names})
    X.to_netcdf('fewsnet_x_global.nc')
else:
    X = xr.open_dataset('fewsnet_x_global.nc')
    X = getattr(X, [i for i in X.data_vars][0])

# download forecast data
if not Path('fewsnet_f_global.nc').is_file() or force_download:
    F = [ dl.download(dl.forecasts[i], i+'.tsv', **args, verbose=True, use_dlauth=False) for i in predictor_names]
    F2 = []
    for i in F:
        i = getattr(i, [j for j in i.data_vars][0])
        i = xc.regrid(i.expand_dims({'M': [0]}), Y.coords['X'].values, Y.coords['Y'].values)
        F2.append(i)
    F = xr.concat(F2, 'M').assign_coords({'M': predictor_names})
    F.to_netcdf('fewsnet_f_global.nc')
else:
    F = xr.open_dataset('fewsnet_f_global.nc')
    F = getattr(F, [i for i in F.data_vars][0])

URL: https://iridl.ldeo.columbia.edu/SOURCES/.UCSB/.CHIRPS/.v2p0/.daily-improved/.global/.0p25/.prcp/92/mul/T/(1%20Jan%201993)/(31%20Dec%202016)/RANGE/T/%28Jun-Aug%201993-2016%29/seasonalAverage/Y/%28-45%29/%2845%29/RANGEEDGES/X/%28-20%29/%2850%29/RANGEEDGES/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv

DOWNLOADING: [*************************] (34268 KB) 0:00:17.902714
URL: https://iridl.ldeo.columbia.edu/SOURCES/.EU/.Copernicus/.CDS/.C3S/.CMCC/.SPSv3p5/.hindcast/.prcp/S/%280000%201%20May%201993-2016%29/VALUES/L/1.5/3.5/RANGEEDGES/%5BL%5D//keepgrids/average/Y/-45/45/RANGEEDGES/X/-20/50/RANGEEDGES/%5BM%5D/average/c%3A/1000/(mm%20m-1)/%3Ac/mul/c%3A/86400/(s%20day-1)/%3Ac/mul/c%3A/92//units/(days)/def/%3Ac/mul/-999/setmissing_value/%5BX/Y%5D%5BL/S/add%5D/cptv10.tsv

DOWNLOADING: [*************************] (2161 KB) 0:00:04.898371
URL: https://iridl.ldeo.columbia.edu/SOURCES/.EU/.Copernicus/.CDS/.C3S/.ECMWF/.SEAS5/.hindcast/.prcp/S/%280000%201%20May%201993-2016%29/VALUES/L/1.5/3.5/RAN

# Climatologies & Drymask

#### Observed Climatology

In [ ]:
yplot = Y.mean('T').plot( subplot_kws={'projection': ccrs.PlateCarree()},  cmap='GnBu')
coast = yplot.axes.coastlines()

#### Model Climatologies

In [ ]:
xplot = X.mean('T').plot(col='M', col_wrap=1, subplot_kws={'projection': ccrs.PlateCarree()},  cmap='GnBu', figsize=(14, 2*len(predictor_names)))
for ax in xplot.axes.flat:
    ax.coastlines()

#### Dry Mask

In [ ]:
drymask = xc.drymask(Y, quantile_threshold=0.66).assign_coords({'C': [predictand_name]})
dplot = drymask.plot(subplot_kws={'projection': ccrs.PlateCarree()})
art = dplot.axes.coastlines()

# Cross Validation

In [ ]:
model_type = xc.cPOELM
crossvalidation_window = 1

kwargs = {
    'ND':5,
    'preprocessing':'minmax',
    'activation':'sigm',
    'hidden_layer_size':5,
    'lat_chunks': 5,
    'lon_chunks': 5,
}

if not client:
    client = Client()

xval = []
i = 1
start = dt.datetime.now()
X, Y = xc.align_chunks(X, Y, 5, 5) 
for x_train, y_train, x_test, y_test in xc.CrossValidator(X, Y, window=crossvalidation_window):
    ohc = xc.RankedTerciles()
    ohc.fit(y_train)
    ohc_y_train = ohc.transform(y_train)
    
    model = model_type(**kwargs)
    model.fit(x_train, ohc_y_train, rechunk=False)
    preds = model.predict_proba(x_test, rechunk=False)
    xval.append(preds.isel(T=crossvalidation_window // 2).load())
    
    print('Finished Fitting Window {} after {}'.format(i, dt.datetime.now() - start))
    i+= 1
hcst = xr.concat(xval, 'T').mean('ND')


# Skill Evaluation
#### Calculate skill

In [ ]:
ohc = xc.RankedTerciles()
ohc.fit(Y)
T = ohc.transform(Y)

rps_fcst = xc.RankProbabilityScore(hcst, T)
rps_climo = xc.RankProbabilityScore(xr.ones_like(hcst) *0.333, T)
rpss = 1 - ( rps_fcst / rps_climo )

groc = xc.GeneralizedROC(hcst, T)


#### Plot Skill

In [ ]:
groc_smooth = xc.gaussian_smooth(groc, kernel=3)
rpss_smooth = xc.gaussian_smooth(rpss, kernel=3)

cmap = plt.get_cmap('autumn_r').copy()
cmap.set_under('lightgray')


fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 12), subplot_kw={'projection': ccrs.PlateCarree()})
groc_smooth.plot(ax=ax[0], vmin=0.5, vmax=1, cmap=cmap)
ax[0].coastlines()
t = ax[0].set_title('Generalized ROC')

rpss_smooth.plot(ax=ax[1], vmin=0, vmax=0.2, cmap=cmap)
ax[1].coastlines()
t = ax[1].set_title('Rank Probability Skill Score')


# Forecasts 

In [ ]:
model = model_type(**kwargs)
model.fit(X, T)
fcst = model.predict_proba(F)
xc.view_probabilistic(xc.gaussian_smooth(fcst.mean('ND'), kernel=3).isel(T=0))